# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [1]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="test-bed-fltk-group16-mb"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Scale the default-node-pool up.

In [2]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [3]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1023 15:48:34.073902    9654 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [4]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1023 15:48:38.441297    9676 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Sun Oct 23 15:48:40 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [19]:
# HOMEWORKS
# EXPERIMENT_FILE="/home/mattiacs/Documents/Developer/Edu/TUDelft/Year_4/Quarter_1/CS4215-QPECS/homeworks_2/ex6_configs/distributed_4_node.json"

# PROJECT
# EXPERIMENT_FILE="../configs/federated_tasks/example_arrival_config.json"
# EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
# CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

EXPERIMENT_FILE="../project/configs/distributed/mnist_fashionmnistcnn/epoch_25/experiment_10e-3_25.json"
CLUSTER_CONFIG="../project/configs/project_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [20]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1023 17:56:41.759289   18561 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled
W1023 17:56:46.696105   18583 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Sun Oct 23 17:56:49 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [21]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1023 18:55:20.055203   26039 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-23-2022 15:57:01 root         INFO     Loading file config/configuration.fltk.json
10-23-2022 15:57:01 root         INFO     Starting in cluster mode.
10-23-2022 15:57:01 root         INFO     Starting with experiment replication: 0 with seed: 42
No argument path is provided.
10-23-2022 15:57:01 root         INFO     Starting as Orchestrator
10-23-2022 15:57:01 root         INFO     Starting Orchestrator, initializing resources....
10-23-2022 15:57:01 root         INFO     Loading in cluster configuration file
10-23-2022 15:57:01 root         INFO     Pointing configuration to in cluster configuration.
10-23-2022 15:57:01 root         INFO     Starting cluster manager
10-23-2022 15:57:01 ClusterManager INFO     Spinning up cluster manag

10-23-2022 16:00:07 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:00:07 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:00:07 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:00:12 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:00:17 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:00:18 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:00:22 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:00:27 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:00:28 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:00:32 root      

10-23-2022 16:04:03 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:04:05 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:04:08 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:04:13 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:04:15 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:04:16 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:04:18 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:04:23 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:04:26 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:04:28 root      

10-23-2022 16:07:59 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:08:04 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:08:04 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:08:09 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:08:14 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:08:14 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:08:19 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:08:24 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:08:24 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 

10-23-2022 16:11:56 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:12:01 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:12:02 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:12:06 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:12:11 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:12:12 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:12:16 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:12:21 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:12:23 ResourceWatchDog INFO     Fetching pod information of 

...

10-23-2022 16:27:45 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:27:50 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:27:55 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:27:56 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:28:00 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:28:05 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:28:06 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:28:06 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:28:10 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 

10-23-2022 16:31:44 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:31:46 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:31:51 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:31:54 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:31:56 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:32:01 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:32:04 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:32:07 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:32:12 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 

10-23-2022 16:35:42 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:35:42 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:35:47 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:35:53 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:35:53 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:35:58 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:36:03 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:36:03 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:36:08 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 

10-23-2022 16:39:39 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:39:41 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:39:44 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:39:49 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:39:51 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:39:54 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:39:59 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:40:01 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:40:04 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 

10-23-2022 16:43:39 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:43:39 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:43:40 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:43:45 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:43:50 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:43:50 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:43:55 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:44:00 root         INFO     Waiting for 3ca1281a-a577-4be9-a4d0-512769d1e1d5 to complete, 0 pending, 0 arrivals
10-23-2022 16:44:00 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 16:44:05 root      

In [22]:
CURRENT_POD_ID="3ca1281a-a577-4be9-a4d0-512769d1e1d5"
# To get logs from learners (example)
kubectl logs -n test trainjob-$CURRENT_POD_ID-master-0

W1023 18:55:41.591398   26095 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-23-2022 15:57:09 root         INFO     Loading file config/configuration.fltk.json
10-23-2022 15:57:09 root         INFO     Starting in client mode
10-23-2022 15:57:09 root         INFO     Starting with host=trainjob-3ca1281a-a577-4be9-a4d0-512769d1e1d5-master-0 and port=23456
10-23-2022 15:57:09 root         INFO     Initializing backend for training process: gloo
10-23-2022 15:57:13 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 0
10-23-2022 15:57:13 torch.distributed.distributed_c10d INFO     Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-23-2022 15:57:13 root         INFO     Starting Creating client with 0
10-23-2022 15:57:13 Client-0-3

10-23-2022 16:43:43 Client-0-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [24,    50] loss: 0.407
10-23-2022 16:44:31 Client-0-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [24,   100] loss: 0.410
10-23-2022 16:44:55 Client-0-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [25,     0] loss: 0.011
10-23-2022 16:45:42 Client-0-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [25,    50] loss: 0.402
10-23-2022 16:46:29 Client-0-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [25,   100] loss: 0.405
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=111239, duration_test=7716, loss_train=1.2798988008499146, accuracy=72.16, loss=21.11841630935669, class_precision=array([0.72953737, 0.87179487, 0.58715596, 0.64465409, 0.56521739,
       0.70909091, 0.42276423, 0.81512605, 0.87727273, 0.88235294]), class_recall=array([0.75925926, 0.8880597 , 0.47583643, 0.75925926, 0.6627451 ,
       0.84782609, 0.23529412, 0.79508197, 0.81779661, 0.94936709]), confusion_mat=array([[

       0.92173913, 0.45248869, 0.8647541 , 0.93644068, 0.95780591]), confusion_mat=array([[206,   0,   7,  18,   3,   2,  29,   0,   5,   0],
       [  0, 256,   1,   7,   2,   0,   0,   0,   2,   0],
       [  5,   1, 181,   3,  40,   1,  36,   0,   2,   0],
       [  6,   4,   6, 230,   9,   1,  14,   0,   0,   0],
       [  1,   0,  33,  12, 187,   0,  20,   0,   2,   0],
       [  0,   0,   0,   2,   0, 212,   0,  12,   1,   3],
       [ 41,   0,  32,  12,  28,   2, 100,   0,   6,   0],
       [  0,   0,   0,   0,   0,  16,   0, 211,   0,  17],
       [  0,   1,   0,   8,   1,   2,   2,   1, 221,   0],
       [  0,   0,   0,   1,   0,   1,   0,   8,   0, 227]])), EpochData(epoch_id=9, num_epochs=26, duration_train=1064736, duration_test=7591, loss_train=0.5443986576795578, accuracy=81.68, loss=11.095644235610962, class_precision=array([0.80232558, 0.98084291, 0.70114943, 0.79109589, 0.7       ,
       0.89830508, 0.50246305, 0.9137931 , 0.92916667, 0.91902834]), class_recall=array(

       0.91176471, 0.53023256, 0.93043478, 0.94142259, 0.93089431]), class_recall=array([0.77037037, 0.95895522, 0.74349442, 0.87777778, 0.75686275,
       0.94347826, 0.5158371 , 0.87704918, 0.95338983, 0.96624473]), confusion_mat=array([[208,   0,   7,  14,   2,   2,  33,   0,   4,   0],
       [  0, 257,   0,   7,   1,   0,   2,   0,   1,   0],
       [  4,   1, 200,   2,  30,   0,  31,   0,   1,   0],
       [  7,   1,   5, 237,   8,   1,  11,   0,   0,   0],
       [  1,   0,  27,  12, 193,   0,  21,   0,   1,   0],
       [  0,   0,   0,   2,   0, 217,   0,   9,   0,   2],
       [ 34,   0,  29,  12,  25,   0, 114,   0,   7,   0],
       [  0,   0,   0,   0,   0,  15,   0, 214,   0,  15],
       [  0,   1,   0,   5,   0,   2,   3,   0, 225,   0],
       [  0,   0,   0,   0,   0,   1,   0,   7,   0, 229]])), EpochData(epoch_id=17, num_epochs=26, duration_train=2019043, duration_test=7394, loss_train=0.454002138376236, accuracy=83.96, loss=9.644444704055786, class_precision=array([

       0.92340426, 0.57345972, 0.93191489, 0.94560669, 0.94238683]), class_recall=array([0.78148148, 0.9738806 , 0.76208178, 0.89259259, 0.76078431,
       0.94347826, 0.54751131, 0.89754098, 0.95762712, 0.96624473]), confusion_mat=array([[211,   0,   6,  12,   2,   2,  33,   0,   4,   0],
       [  0, 261,   0,   5,   0,   0,   1,   0,   1,   0],
       [  6,   1, 205,   2,  26,   0,  28,   0,   1,   0],
       [  6,   1,   6, 241,   8,   0,   8,   0,   0,   0],
       [  1,   0,  30,  12, 194,   0,  18,   0,   0,   0],
       [  0,   0,   0,   1,   0, 217,   0,   9,   0,   3],
       [ 33,   0,  26,   8,  26,   0, 121,   0,   7,   0],
       [  0,   0,   0,   0,   0,  14,   0, 219,   0,  11],
       [  0,   1,   0,   5,   1,   1,   2,   0, 226,   0],
       [  0,   0,   0,   0,   0,   1,   0,   7,   0, 229]])), EpochData(epoch_id=25, num_epochs=26, duration_train=2971834, duration_test=7609, loss_train=0.40493588864803315, accuracy=84.96, loss=8.888771444559097, class_precision=array

In [23]:
# To get logs from learners (federated learning)
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-0

W1023 18:56:24.362622   26157 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-23-2022 15:57:11 root         INFO     Loading file config/configuration.fltk.json
10-23-2022 15:57:11 root         INFO     Starting in client mode
10-23-2022 15:57:11 root         INFO     Starting with host=trainjob-3ca1281a-a577-4be9-a4d0-512769d1e1d5-master-0 and port=23456
10-23-2022 15:57:11 root         INFO     Initializing backend for training process: gloo
10-23-2022 15:57:13 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 1
10-23-2022 15:57:13 torch.distributed.distributed_c10d INFO     Rank 1: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-23-2022 15:57:13 root         INFO

10-23-2022 16:42:55 Client-1-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [24,     0] loss: 0.011
10-23-2022 16:43:43 Client-1-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [24,    50] loss: 0.407
10-23-2022 16:44:31 Client-1-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [24,   100] loss: 0.410
10-23-2022 16:44:55 Client-1-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [25,     0] loss: 0.011
10-23-2022 16:45:42 Client-1-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [25,    50] loss: 0.402
10-23-2022 16:46:29 Client-1-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [25,   100] loss: 0.405
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=111287, duration_test=7999, loss_train=1.2798988008499146, accuracy=70.8, loss=20.770262360572815, class_precision=array([0.71804511, 0.85501859, 0.53744493, 0.62012987, 0.51953125,
       0.73913043, 0.48387097, 0.75875486, 0.84716157, 0.87924528]), class_recall=array([0.80252101, 0.85501859, 0.51046025, 0.74319066, 0.57

       0.90298507, 0.62162162, 0.85433071, 0.91286307, 0.90769231]), class_recall=array([0.80672269, 0.91821561, 0.76150628, 0.84824903, 0.68695652,
       0.90298507, 0.54330709, 0.85098039, 0.93220339, 0.92913386]), confusion_mat=array([[192,   0,   4,  17,   0,   1,  19,   0,   5,   0],
       [  0, 247,   6,  12,   2,   0,   2,   0,   0,   0],
       [  1,   0, 182,   3,  29,   1,  22,   0,   1,   0],
       [  6,   7,   2, 218,   5,   1,  16,   0,   2,   0],
       [  0,   1,  35,  12, 158,   0,  22,   0,   2,   0],
       [  0,   0,   0,   0,   0, 242,   0,  19,   2,   5],
       [ 45,   0,  28,  11,  24,   1, 138,   0,   7,   0],
       [  0,   0,   0,   0,   0,  18,   0, 217,   1,  19],
       [  1,   0,   2,   4,   1,   3,   3,   2, 220,   0],
       [  0,   0,   0,   0,   0,   1,   0,  16,   1, 236]])), EpochData(epoch_id=9, num_epochs=26, duration_train=1064782, duration_test=7996, loss_train=0.5443986576795578, accuracy=82.4, loss=10.581998020410538, class_precision=array([

       0.95      , 0.63829787, 0.87596899, 0.91803279, 0.91221374]), class_recall=array([0.81092437, 0.92565056, 0.79079498, 0.85992218, 0.71304348,
       0.92164179, 0.59055118, 0.88627451, 0.94915254, 0.94094488]), confusion_mat=array([[193,   0,   4,  13,   0,   1,  22,   0,   5,   0],
       [  1, 249,   5,  10,   1,   0,   3,   0,   0,   0],
       [  2,   0, 189,   2,  26,   0,  20,   0,   0,   0],
       [  5,   9,   1, 221,   4,   0,  14,   0,   3,   0],
       [  0,   0,  30,  10, 164,   0,  24,   0,   2,   0],
       [  0,   0,   0,   0,   0, 247,   0,  16,   1,   4],
       [ 41,   0,  23,  11,  21,   1, 150,   0,   7,   0],
       [  0,   0,   0,   0,   0,   9,   0, 226,   1,  19],
       [  0,   0,   2,   3,   1,   2,   2,   2, 224,   0],
       [  0,   0,   0,   0,   0,   0,   0,  14,   1, 239]])), EpochData(epoch_id=17, num_epochs=26, duration_train=2019089, duration_test=8295, loss_train=0.454002138376236, accuracy=84.2, loss=9.084617137908936, class_precision=array([0

       0.95785441, 0.66129032, 0.88888889, 0.93004115, 0.93023256]), class_recall=array([0.81512605, 0.9330855 , 0.80753138, 0.88326848, 0.72608696,
       0.93283582, 0.64566929, 0.90980392, 0.95762712, 0.94488189]), confusion_mat=array([[194,   0,   3,  13,   0,   1,  25,   0,   2,   0],
       [  1, 251,   3,   9,   1,   0,   4,   0,   0,   0],
       [  2,   0, 193,   3,  23,   0,  18,   0,   0,   0],
       [  3,   6,   1, 227,   4,   0,  14,   0,   2,   0],
       [  0,   0,  28,  11, 167,   0,  21,   0,   3,   0],
       [  0,   0,   0,   0,   0, 250,   0,  14,   1,   3],
       [ 35,   0,  17,   9,  21,   1, 164,   0,   7,   0],
       [  0,   0,   0,   0,   0,   7,   0, 232,   1,  15],
       [  0,   0,   2,   1,   1,   2,   2,   2, 226,   0],
       [  0,   0,   0,   0,   0,   0,   0,  13,   1, 240]])), EpochData(epoch_id=25, num_epochs=26, duration_train=2971880, duration_test=7704, loss_train=0.40493588864803315, accuracy=85.8, loss=8.316127091646194, class_precision=array(

In [24]:
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-1

W1023 18:56:32.534616   26216 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-23-2022 15:57:13 root         INFO     Loading file config/configuration.fltk.json
10-23-2022 15:57:13 root         INFO     Starting in client mode
10-23-2022 15:57:13 root         INFO     Starting with host=trainjob-3ca1281a-a577-4be9-a4d0-512769d1e1d5-master-0 and port=23456
10-23-2022 15:57:13 root         INFO     Initializing backend for training process: gloo
10-23-2022 15:57:13 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 2
10-23-2022 15:57:13 torch.distributed.distributed_c10d INFO     Rank 2: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-23-2022 15:57:13 root         INFO

10-23-2022 16:42:55 Client-2-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [24,     0] loss: 0.011
10-23-2022 16:43:43 Client-2-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [24,    50] loss: 0.407
10-23-2022 16:44:31 Client-2-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [24,   100] loss: 0.410
10-23-2022 16:44:55 Client-2-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [25,     0] loss: 0.011
10-23-2022 16:45:42 Client-2-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [25,    50] loss: 0.402
10-23-2022 16:46:29 Client-2-3ca1281a-a577-4be9-a4d0-512769d1e1d5 INFO     [25,   100] loss: 0.405
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=111290, duration_test=8188, loss_train=1.2798988008499146, accuracy=70.56, loss=21.500498831272125, class_precision=array([0.72307692, 0.81081081, 0.5106383 , 0.60616438, 0.52649007,
       0.69784173, 0.48543689, 0.80620155, 0.88888889, 0.89655172]), class_recall=array([0.7704918 , 0.91304348, 0.51282051, 0.70238095, 0.6

       0.8879668 , 0.5787234 , 0.88715953, 0.91544118, 0.93280632]), class_recall=array([0.75409836, 0.94782609, 0.72222222, 0.86904762, 0.69348659,
       0.90677966, 0.50557621, 0.8976378 , 0.91881919, 0.94779116]), confusion_mat=array([[184,   0,   3,  20,   3,   3,  27,   0,   4,   0],
       [  0, 218,   1,   7,   2,   0,   1,   0,   1,   0],
       [  1,   0, 169,   6,  36,   0,  19,   0,   3,   0],
       [  1,   1,   4, 219,  10,   0,  17,   0,   0,   0],
       [  0,   0,  34,  15, 181,   0,  29,   0,   2,   0],
       [  0,   0,   0,   0,   0, 214,   0,  17,   1,   4],
       [ 39,   1,  32,  14,  31,   4, 136,   0,  12,   0],
       [  0,   0,   0,   0,   0,  14,   0, 228,   0,  12],
       [  1,   0,   1,   6,   1,   3,   6,   3, 249,   1],
       [  0,   0,   0,   1,   0,   3,   0,   9,   0, 236]])), EpochData(epoch_id=9, num_epochs=26, duration_train=1064787, duration_test=7392, loss_train=0.5443986576795578, accuracy=81.8, loss=11.242806643247604, class_precision=array([

       [  0,   0,   0,   1,   0,   3,   0,  10,   0, 235]])), EpochData(epoch_id=16, num_epochs=26, duration_train=1901390, duration_test=7892, loss_train=0.4620594757795334, accuracy=83.32, loss=9.860204041004181, class_precision=array([0.80888889, 1.        , 0.73092369, 0.77385159, 0.72413793,
       0.91525424, 0.62605042, 0.89353612, 0.92028986, 0.951417  ]), class_recall=array([0.74590164, 0.96521739, 0.77777778, 0.86904762, 0.72413793,
       0.91525424, 0.55390335, 0.92519685, 0.93726937, 0.9437751 ]), confusion_mat=array([[182,   0,   4,  21,   1,   2,  30,   0,   4,   0],
       [  0, 222,   0,   4,   2,   0,   1,   0,   1,   0],
       [  1,   0, 182,   5,  31,   0,  13,   0,   2,   0],
       [  1,   0,   7, 219,  12,   0,  12,   0,   1,   0],
       [  0,   0,  26,  17, 189,   0,  28,   0,   1,   0],
       [  0,   0,   0,   0,   0, 216,   0,  16,   1,   3],
       [ 39,   0,  29,  12,  25,   3, 149,   0,  12,   0],
       [  0,   0,   0,   0,   0,  11,   0, 235,   0,   8]

       [  0,   0,   0,   0,   0, 220,   0,  13,   0,   3],
       [ 35,   0,  26,  13,  25,   2, 157,   0,  11,   0],
       [  0,   0,   0,   0,   0,  11,   0, 236,   0,   7],
       [  2,   0,   0,   3,   1,   1,   5,   2, 256,   1],
       [  0,   0,   0,   0,   0,   3,   1,  10,   0, 235]])), EpochData(epoch_id=24, num_epochs=26, duration_train=2852481, duration_test=8104, loss_train=0.40996941685676574, accuracy=84.56, loss=9.019066572189331, class_precision=array([0.8280543 , 0.99555556, 0.75510204, 0.79003559, 0.74806202,
       0.92050209, 0.64112903, 0.90769231, 0.92753623, 0.95546559]), class_recall=array([0.75      , 0.97391304, 0.79059829, 0.88095238, 0.7394636 ,
       0.93220339, 0.59107807, 0.92913386, 0.94464945, 0.94779116]), confusion_mat=array([[183,   0,   5,  19,   2,   2,  29,   0,   4,   0],
       [  0, 224,   0,   3,   2,   0,   0,   0,   1,   0],
       [  2,   0, 185,   6,  25,   0,  14,   0,   2,   0],
       [  1,   1,   7, 222,  10,   0,  10,   0,   1,   0

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [25]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ../logging

W1023 19:03:06.867426   26766 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1023 19:03:07.651839   26780 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [26]:
helm uninstall -n test experiment-orchestrator

W1023 19:03:18.736456   26811 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [27]:
helm uninstall extractor -n test

W1023 19:03:46.312453   26913 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [28]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1023 19:05:56.705875   27066 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
pytorchjob.kubeflow.org "trainjob-3ca1281a-a577-4be9-a4d0-512769d1e1d5" deleted
pytorchjob.kubeflow.org "trainjob-8c75fbb2-7172-46a3-b9f2-64fd122ca38d" deleted
pytorchjob.kubeflow.org "trainjob-9a7ddfd7-6eee-4694-acfb-f923de6bee62" deleted
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].
